In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pyblock2.driver.core import DMRGDriver, SymmetryTypes, MPOAlgorithmTypes, Block2Wrapper

# My Code for Bose Hubbard Model

In [9]:
### Using CustomHamiltonian Example

L = 2
U = 2
N_ELEC = 2
TWO_SZ = 0
n_max = 3

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny, n_threads=4)

driver.set_symmetry_groups("U1")
Q = driver.bw.SX

In [10]:
# [Part A] Set states and matrix representation of operators in local Hilbert space
site_basis, site_ops = [], []

for k in range(L):
    basis = [(Q(2, 0), 1), (Q(1, 1), 1), (Q(2, 0), 1)] # [0ab2]
    ops = {
        "": np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]),   # identity
        "c": np.array([[0, 0, 0, 0], [1, 0, 0, 0], [0, np.sqrt(2), 0, 0], [0, 0, np.sqrt(3), 0]]),  # alpha+
        "d": np.array([[0, 1, 0, 0], [0, 0, np.sqrt(2), 0], [0, 0, 0, np.sqrt(3)], [0, 0, 0, 0]]),  # alpha
    }
    site_basis.append(basis)
    site_ops.append(ops)

AssertionError: 

In [11]:
# [Part B] Set Hamiltonian terms
driver.initialize_system(n_sites=L, vacuum=Q(0, 0), target=Q(n_max, n_max), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

b.add_term("cd", np.array([[i, i + 1, i + 1, i] for i in range(L - 1)]).ravel(), -1)
b.add_term("cd", np.array([i for i in range(L) for _ in range(4)]), U)

AssertionError: 

In [12]:
# [Part C] Perform DMRG
mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops="cd"), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[250] * 4 + [500] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=1)
# print("DMRG energy = %20.15f" % energy)

NameError: name 'b' is not defined